# Matrices de Transición (Bigramas) en Hipnogramas del Sueño

Este notebook analiza las **transiciones entre fases del sueño** mediante matrices de transición (bigramas).

## Objetivos

1. **Agrupar fases**: S3 y S4 se agrupan en un solo símbolo (S3/S4) para limpiar la matriz, igual que en los otros notebooks.
2. **Matriz Normal**: Matriz de transición estándar con probabilidades condicionales.
3. **Matriz Sin Diagonal**: Versión donde la probabilidad de quedarse en la misma fase (diagonal) es cero, re-normalizada por fila.
   - **Objetivo**: Ver claramente hacia dónde se va el cerebro cuando decide abandonar una fase, sin que el gráfico se "coma" el color por la alta probabilidad de quedarse en la misma fase.


In [177]:
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [178]:
# Configuración (consistente con fase2_dinamica.ipynb y ventanasDeslizantes.ipynb)
RUTA_DATOS = Path("../Datos")

ETIQUETAS_FASES = {
    0: "Vigilia (W)",
    1: "S1",
    2: "S2",
    3: "S3/S4",
    4: "REM",
    5: "Movimiento/SC"
}

def agrupar_fase(valor):
    """Mapea cualquier fase cruda al alfabeto reducido."""
    return {4: 3, 5: 4, 6: 5, 7: 5}.get(valor, valor)


def remapear_secuencia(secuencia):
    """Aplica `agrupar_fase` a cada época para unificar símbolos antes del conteo."""
    return [agrupar_fase(valor) for valor in secuencia]

print("Configuración cargada.")


Configuración cargada.


In [179]:
def listar_pacientes(ruta_datos=None):
    """Devuelve los IDs disponibles (archivos `Scoring_*_2`)."""
    ruta_datos = ruta_datos or RUTA_DATOS
    pacientes = []
    for archivo in ruta_datos.iterdir():
        nombre = archivo.name
        if archivo.is_file() and nombre.startswith("Scoring_") and nombre.endswith("_2"):
            pacientes.append(nombre.replace("Scoring_", ""))
    pacientes.sort()
    if not pacientes:
        raise RuntimeError("No se encontraron archivos `Scoring_*_2` en la ruta indicada.")
    return pacientes


def cargar_hipnograma(paciente, ruta_datos=None):
    """Carga la columna `scor_clean` y la devuelve como lista de enteros."""
    ruta_datos = ruta_datos or RUTA_DATOS
    ruta_archivo = ruta_datos / f"Scoring_{paciente}"
    columnas = ["epoca", "scor_clean", "scor_quasi"]
    df = pd.read_csv(ruta_archivo, sep=r"\s+", header=None, names=columnas, dtype=int)
    return df["scor_clean"].tolist()

print("Funciones de carga definidas.")


Funciones de carga definidas.


In [180]:
# Cargar todos los hipnogramas y aplicar agrupación
pacientes = listar_pacientes()
print(f"Pacientes detectados ({len(pacientes)}): {', '.join(pacientes)}")

hipnogramas_agrupados = {}
for paciente in pacientes:
    raw_seq = cargar_hipnograma(paciente)
    seq_agrupada = remapear_secuencia(raw_seq)
    hipnogramas_agrupados[paciente] = seq_agrupada

print(f"\nTotal de épocas procesadas: {sum(len(seq) for seq in hipnogramas_agrupados.values())}")
print("\nAlfabeto agrupado usado:")
for codigo, etiqueta in ETIQUETAS_FASES.items():
    print(f"  {codigo}: {etiqueta}")


Pacientes detectados (10): AR_2, DG_2, EL_2, GA_2, IN_2, JS_2, LL_2, SS_2, VB_2, VC_2

Total de épocas procesadas: 9501

Alfabeto agrupado usado:
  0: Vigilia (W)
  1: S1
  2: S2
  3: S3/S4
  4: REM
  5: Movimiento/SC


In [181]:
def extraer_bigramas(secuencia):
    """Extrae todos los bigramas (pares consecutivos) de una secuencia."""
    bigramas = []
    for i in range(len(secuencia) - 1):
        bigramas.append((secuencia[i], secuencia[i+1]))
    return bigramas

# Extraer bigramas de todos los pacientes
todos_los_bigramas = []
for paciente, secuencia in hipnogramas_agrupados.items():
    bigramas = extraer_bigramas(secuencia)
    todos_los_bigramas.extend(bigramas)

print(f"Total de bigramas extraídos: {len(todos_los_bigramas)}")
print(f"Ejemplos de bigramas: {todos_los_bigramas[:10]}")


Total de bigramas extraídos: 9491
Ejemplos de bigramas: [(5, 5), (5, 5), (5, 5), (5, 5), (5, 5), (5, 5), (5, 5), (5, 5), (5, 5), (5, 5)]


In [182]:
def construir_matriz_transicion(bigramas):
    """
    Construye la matriz de transición (probabilidades condicionales) a partir de bigramas.
    
    Returns:
        matriz_absoluta: DataFrame con conteos absolutos
        matriz_probabilidades: DataFrame con probabilidades condicionales (normalizada por fila)
    """
    # Obtener todas las fases únicas presentes en los bigramas
    fases_unicas = sorted(set([fase for bigrama in bigramas for fase in bigrama]))
    
    # Inicializar matriz de conteos absolutos
    matriz_absoluta = pd.DataFrame(
        0, 
        index=fases_unicas, 
        columns=fases_unicas, 
        dtype=int
    )
    
    # Contar bigramas
    contador = Counter(bigramas)
    for (desde, hacia), count in contador.items():
        if desde in matriz_absoluta.index and hacia in matriz_absoluta.columns:
            matriz_absoluta.loc[desde, hacia] = count
    
    # Calcular probabilidades condicionales (normalizar por fila)
    suma_por_fila = matriz_absoluta.sum(axis=1)
    matriz_probabilidades = matriz_absoluta.div(suma_por_fila, axis=0).fillna(0)
    
    return matriz_absoluta, matriz_probabilidades

# Construir matriz de transición
matriz_abs, matriz_prob = construir_matriz_transicion(todos_los_bigramas)

print("Matriz de transición construida.")
print(f"\nDimensiones: {matriz_prob.shape}")
print(f"\nFases presentes: {sorted(matriz_prob.index.tolist())}")
print("\nMatriz de probabilidades (primeras filas):")
print(matriz_prob.head())


Matriz de transición construida.

Dimensiones: (6, 6)

Fases presentes: [0, 1, 2, 3, 4, 5]

Matriz de probabilidades (primeras filas):
          0         1         2         3         4         5
0  0.689655  0.172414  0.074713  0.000000  0.000000  0.063218
1  0.064846  0.467577  0.426621  0.000000  0.006826  0.034130
2  0.005815  0.007381  0.887050  0.052785  0.016551  0.030418
3  0.001509  0.000503  0.095573  0.879276  0.001509  0.021630
4  0.005305  0.003183  0.022281  0.000531  0.944828  0.023873


In [183]:
def construir_matriz_sin_diagonal(matriz_probabilidades):
    """
    Construye una versión de la matriz donde la diagonal (quedarse en la misma fase) es cero,
    y luego re-normaliza por fila.
    
    Objetivo: Ver claramente hacia dónde se va el cerebro cuando decide abandonar una fase,
    sin que el gráfico se 'coma' el color por la alta probabilidad de quedarse en la misma fase.
    """
    # Copiar la matriz
    matriz_sin_diag = matriz_probabilidades.copy()
    
    # Poner la diagonal en cero
    for fase in matriz_sin_diag.index:
        if fase in matriz_sin_diag.columns:
            matriz_sin_diag.loc[fase, fase] = 0
    
    # Re-normalizar por fila (suma de cada fila debe ser 1)
    suma_por_fila = matriz_sin_diag.sum(axis=1)
    # Evitar división por cero (si una fila tiene suma 0, mantenerla en 0)
    matriz_sin_diag = matriz_sin_diag.div(suma_por_fila, axis=0).fillna(0)
    
    return matriz_sin_diag

# Construir matriz sin diagonal
matriz_sin_diag = construir_matriz_sin_diagonal(matriz_prob)

print("Matriz sin diagonal construida.")
print("\nMatriz sin diagonal (primeras filas):")
print(matriz_sin_diag.head())


Matriz sin diagonal construida.

Matriz sin diagonal (primeras filas):
          0         1         2         3         4         5
0  0.000000  0.555556  0.240741  0.000000  0.000000  0.203704
1  0.121795  0.000000  0.801282  0.000000  0.012821  0.064103
2  0.051485  0.065347  0.000000  0.467327  0.146535  0.269307
3  0.012500  0.004167  0.791667  0.000000  0.012500  0.179167
4  0.096154  0.057692  0.403846  0.009615  0.000000  0.432692


In [184]:
def graficar_heatmap(matriz_probabilidades, titulo, mostrar_diagonal=True):
    """
    Genera un heatmap interactivo con Plotly para la matriz de transición.
    
    Args:
        matriz_probabilidades: DataFrame con probabilidades condicionales
        titulo: Título del gráfico
        mostrar_diagonal: Si False, oculta visualmente la diagonal (poner NaN)
    """
    # Preparar etiquetas
    etiquetas_filas = [ETIQUETAS_FASES.get(fase, f"Fase {fase}") for fase in matriz_probabilidades.index]
    etiquetas_columnas = [ETIQUETAS_FASES.get(fase, f"Fase {fase}") for fase in matriz_probabilidades.columns]
    
    # Preparar valores y textos
    valores = matriz_probabilidades.values
    # Mostrar todos los valores, incluyendo los que son exactamente 0
    textos = [[f"{valor:.3f}" for valor in fila] for fila in valores]
    
    # Si no queremos mostrar la diagonal, asegurarnos de que el texto esté vacío
    # pero mantener el valor 0 para que se vea con el color apropiado del heatmap
    if not mostrar_diagonal:
        valores_plot = valores.copy()
        for i, fase in enumerate(matriz_probabilidades.index):
            if fase in matriz_probabilidades.columns:
                j = matriz_probabilidades.columns.get_loc(fase)
                # No poner NaN, dejar 0 pero sin texto
                textos[i][j] = ""
    else:
        valores_plot = valores
    
    # Crear heatmap
    fig = go.Figure(
        data=go.Heatmap(
            z=valores_plot,
            x=etiquetas_columnas,
            y=etiquetas_filas,
            text=textos,
            texttemplate="%{text}",
            textfont=dict(size=11, color="white"),
            colorscale="Cividis",
            zmin=0,
            zmax=1,
            colorbar=dict(
                title=dict(text="Probabilidad", font=dict(color="white")),
                tickfont=dict(color="white")
            ),
            hovertemplate="<b>Desde:</b> %{y}<br><b>Hacia:</b> %{x}<br><b>Probabilidad:</b> %{z:.4f}<extra></extra>"
        )
    )
    
    fig.update_layout(
        template="plotly_dark",
        paper_bgcolor="black",
        plot_bgcolor="black",
        title=dict(
            text=f"<b>{titulo}</b><br><sub>P(Fase siguiente | Fase actual)</sub>",
            font=dict(size=16, color="white"),
            x=0.5
        ),
        xaxis_title="Fase siguiente",
        yaxis_title="Fase actual",
        width=900,
        height=700,
        xaxis=dict(
            title=dict(font=dict(color="white")),
            tickfont=dict(color="white")
        ),
        yaxis=dict(
            title=dict(font=dict(color="white")),
            tickfont=dict(color="white"),
            autorange="reversed"  # Invertir eje Y para que la diagonal quede de arriba-izq a abajo-der
        )
    )
    
    fig.show()

print("Función de visualización definida.")


Función de visualización definida.


## Versión 1: Matriz de Transición Normal

Esta matriz muestra las probabilidades condicionales completas, incluyendo la probabilidad de quedarse en la misma fase (diagonal).


In [185]:
graficar_heatmap(
    matriz_prob, 
    "Matriz de Transición (Bigramas) - Versión Normal",
    mostrar_diagonal=True
)

## Versión 2: Matriz Sin Diagonal

Esta versión elimina la probabilidad de quedarse en la misma fase (diagonal = 0) y re-normaliza por fila. Esto permite ver claramente **hacia dónde se va el cerebro cuando decide abandonar una fase**, sin que el gráfico se "coma" el color por la alta probabilidad de quedarse en la misma fase.


In [186]:
graficar_heatmap(
    matriz_sin_diag, 
    "Matriz de Transición (Bigramas) - Versión Sin Diagonal",
    mostrar_diagonal=False  # Ocultar visualmente la diagonal (aunque ya es 0)
)


## Análisis de Transiciones

A continuación se muestran algunas estadísticas sobre las transiciones más frecuentes.


In [187]:
# Mostrar las transiciones más frecuentes
contador_bigramas = Counter(todos_los_bigramas)

df_transiciones = pd.DataFrame([
    {
        "Desde": ETIQUETAS_FASES.get(desde, f"Fase {desde}"),
        "Hacia": ETIQUETAS_FASES.get(hacia, f"Fase {hacia}"),
        "Frecuencia": count,
        "Probabilidad": matriz_prob.loc[desde, hacia]
    }
    for (desde, hacia), count in contador_bigramas.most_common(20)
])

print("Top 20 transiciones más frecuentes:")
print(df_transiciones.to_string(index=False))

# Mostrar estadísticas por fase de origen
print("\nEstadísticas por fase de origen:")


for fase in sorted(matriz_prob.index):
    etiqueta = ETIQUETAS_FASES.get(fase, f"Fase {fase}")
    fila = matriz_prob.loc[fase]
    # Encontrar el destino más probable (excluyendo la diagonal si queremos ver transiciones reales)
    fila_sin_diag = fila.copy()
    fila_sin_diag.loc[fase] = 0  # Excluir quedarse en la misma fase
    if fila_sin_diag.sum() > 0:
        fila_sin_diag = fila_sin_diag / fila_sin_diag.sum()  # Re-normalizar
        destino_mas_probable = fila_sin_diag.idxmax()
        prob_destino = fila_sin_diag.loc[destino_mas_probable]
        etiqueta_destino = ETIQUETAS_FASES.get(destino_mas_probable, f"Fase {destino_mas_probable}")
        prob_quedarse = matriz_prob.loc[fase, fase]
        print(f"\n{etiqueta}:")
        print(f"  Probabilidad de quedarse: {prob_quedarse:.3f}")
        print(f"  Destino más probable (al abandonar): {etiqueta_destino} ({prob_destino:.3f})")


Top 20 transiciones más frecuentes:
        Desde         Hacia  Frecuencia  Probabilidad
           S2            S2        3966      0.887050
          REM           REM        1781      0.944828
        S3/S4         S3/S4        1748      0.879276
Movimiento/SC Movimiento/SC         253      0.500000
  Vigilia (W)   Vigilia (W)         240      0.689655
           S2         S3/S4         236      0.052785
        S3/S4            S2         190      0.095573
           S1            S1         137      0.467577
           S2 Movimiento/SC         136      0.030418
           S1            S2         125      0.426621
Movimiento/SC            S2         121      0.239130
           S2           REM          74      0.016551
  Vigilia (W)            S1          60      0.172414
Movimiento/SC            S1          56      0.110672
Movimiento/SC   Vigilia (W)          48      0.094862
          REM Movimiento/SC          45      0.023873
        S3/S4 Movimiento/SC          43      0

## Conclusiones Biológicas (Interpretación de los Datos)

Basado en los números de la matriz "Sin Diagonal", estas son las conclusiones que se pueden extraer:


### S2 es el "Hub" o Distribuidor Universal

Observa la fila de S2 en la matriz sin diagonal:
- **46.7%** se va a S3/S4 (Sueño Profundo).
- **14.7%** se va a REM.
- **26.9%** se va a Movimiento/SC.

**Conclusión:** La función principal de la fase 2 es servir de puerta de entrada al sueño profundo. Es casi 3 veces más probable profundizar el sueño (ir a S3) que entrar a soñar (ir a REM) desde S2.


### El "Ciclo de Retorno" del Sueño Profundo

Observa la fila de S3/S4:
- **79.2%** se regresa a S2.
- Casi **0%** se va a REM o Vigilia directamente.

**Conclusión:** El sueño profundo es un "callejón sin salida" fisiológico. Para salir de él, el cerebro debe aligerarse primero hacia S2. No se salta de S3 a REM. Esto valida la estructura cíclica clásica (N1 $\to$ N2 $\to$ N3 $\to$ N2 $\to$ REM).


### La Fragilidad del REM

Observa la fila de REM:
- **43.3%** se va a Movimiento/SC (Despertares breves o artefactos).
- **40.4%** se regresa a S2.

**Conclusión:** Aunque el REM es muy estable (matriz normal), cuando termina, suele ser un final "abrupto" o inestable, resultando frecuentemente en movimiento o micro-despertares, antes de volver a caer en sueño ligero (S2).
